In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !if [ -d deepul ]; then rm -Rf deepul; fi
    !git clone -b magda https://github.com/mgswiss15/deepul.git 
    !unzip -qq deepul/homeworks/hw1/data/hw1_data.zip -d deepul/homeworks/hw1/data/
    !pip install ./deepul
else:
    %load_ext autoreload
    %autoreload 2
    import sys
    import pathlib
    cwd = pathlib.Path().absolute()
    sys.path.append(str(cwd.parents[0]))
    sys.path.append(str(cwd.parents[1]))
    print(sys.path)

['/home/magda/Github/deepul/homeworks/hw2', '/home/magda/anaconda3/envs/pytorch/lib/python38.zip', '/home/magda/anaconda3/envs/pytorch/lib/python3.8', '/home/magda/anaconda3/envs/pytorch/lib/python3.8/lib-dynload', '', '/home/magda/anaconda3/envs/pytorch/lib/python3.8/site-packages', '/home/magda/anaconda3/envs/pytorch/lib/python3.8/site-packages/IPython/extensions', '/home/magda/.ipython', '/home/magda/Github/deepul/homeworks', '/home/magda/Github/deepul']


In [2]:
from deepul.notebook2script import *

# Training structure

In [3]:
#export
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

def training(train_loader, test_loader, model, optim_algo, learn_rate, device=torch.device('cpu'), epochs=20, grad_clip=None, warmup=200):
    nll_train = []
    nll_test = []
    optimizer = optim_algo(model.parameters(), learn_rate)
    print('First eval')
    test_loss = evaluate(test_loader, model, device)
    nll_test.append(test_loss)
    print('Training loop')
    for epoch in range(epochs):
        loss_list = train(train_loader, model, optimizer, learn_rate, device, grad_clip=grad_clip, warmup=warmup)
        nll_train += loss_list
        test_loss = evaluate(test_loader, model, device)
        nll_test.append(test_loss)
        if epoch%10. == 0.:
            print(f'Epoch {epoch} loss train: {loss_list[-1]}, test: {nll_test[-1]}')
    return nll_train, nll_test
        
def train(train_loader, model, optimizer, learn_rate, device, grad_clip=None, warmup=200):
    """For back compatibility batch can be (input) or (input, output)"""
    model.train()
    loss_list = []
    for i, batch in enumerate(train_loader):
        lr = min(1, 1/warmup * i) * learn_rate
        optimizer.lr = lr
        if isinstance(batch, list):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            batch = [x, y]
        else:
            batch = batch.to(device).requires_grad_()
        latent, log_det_jacobian = model(batch)
        loss = model.loss_function(latent, log_det_jacobian) 
        optimizer.zero_grad()
        loss.backward()
        if grad_clip:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        loss_list.append(loss.item())
    return loss_list

def evaluate(test_loader, model, device):
    '''Evaluate without no_grad'''
    model.eval()
    with torch.no_grad():
        loss = 0
        for batch in test_loader:
            if isinstance(batch, list):
                x, y = batch
                n_inst = x.shape[0]
                x = x.to(device)
                y = y.to(device)
                batch = [x, y]
            else:
                n_inst = batch.shape[0]
                batch = batch.to(device)
            latent, log_det_jacobian = model(batch)
            loss += model.loss_function(latent, log_det_jacobian).item() * n_inst
    total_loss = loss / len(test_loader.dataset)
    return total_loss
        
def get_loaders(train_data, test_data, bs):
    """For back compatibility works with datasets and numpy arrays as inputs"""
    if isinstance(train_data, np.ndarray):
        train_data = torch.as_tensor(train_data, dtype=torch.float)
        test_data = torch.as_tensor(test_data, dtype=torch.float)
    assert (isinstance(train_data, data.Dataset) | isinstance(train_data, torch.Tensor)), print(f'train_data are not in good format (np.ndarray, torch.Tensor or data.Dataset)')
    train_loader = data.DataLoader(train_data, batch_size=bs, shuffle=True)
    test_loader = data.DataLoader(test_data, batch_size=bs, shuffle=False)
    return train_loader, test_loader

### Export

In [4]:
nb2script('Utils2')

Converted Utils2.ipynb to /home/magda/Github/deepul/deepul/exp_utils2.py
